In [3]:
import os
import pandas as pd



---
 # اکسترکت یا استخراج فایل ها

In [ ]:
#برای خارج کردن از حالت زیپ کدهای زیر را در ترمینال مینویسیم

# tar -xvjf 20021010_easy_ham.tar.bz2
#توجه که 20021010_easy_ham.tar.bz2 اسم خود فایل است

# tar -xvjf 20021010_spam.tar.bz2

---
# خواندن فایل ها

In [4]:
#خواندن فایل -----------
def load_emails_from_folder(folder):
    emails = []
    for filename in os.listdir(folder):
    # یه پوشه داریم به اسم spam/ که پر از فایل ایمیله.
    # os.listdir(folder) مثل اینه که بری داخل اون پوشه و اسم همه‌ی فایل‌ها رو بخونی.

        path = os.path.join(folder, filename)
        # فرض کن داری با فولدر spam/ کار می‌کنی.
        # برای خوندن فایل، نیاز به آدرس کاملش داری
        # مزیتش اینه که روی همه‌ی سیستم‌عامل‌ها (ویندوز، لینوکس، مک) درست کار می‌کنه.

        with open(path, 'r', encoding='latin-1') as f:
        # encoding='latin-1' → چون بعضی ایمیل‌ها کاراکترهای خاص دارن که با utf-8 باز نمی‌شن.
            emails.append(f.read())
    return emails

spam_emails = load_emails_from_folder('data/spam')
ham_emails = load_emails_from_folder('data/easy_ham')

print(f"Loaded {len(spam_emails)} spam emails")
print(f"Loaded {len(ham_emails)} ham emails")


Loaded 501 spam emails
Loaded 2551 ham emails


---
# دید کلی از دیتاست

In [8]:
import pandas as pd

# ایجاد دیتافریم برای ایمیل‌های هرزنامه
spam_df = pd.DataFrame({'text': spam_emails, 'label': 1}) # 1 برای هرزنامه

# ایجاد دیتافریم برای ایمیل‌های غیر هرزنامه
ham_df = pd.DataFrame({'text': ham_emails, 'label': 0}) # 0 برای غیر هرزنامه

# ترکیب دو دیتافریم
df = pd.concat([spam_df, ham_df], ignore_index=True)

print("نمایش چند ردیف اول دیتافریم ترکیب شده:")
print(df.head())

print("\nاطلاعات کلی دیتافریم ترکیب شده:")
df.info()

print("\nتعداد ایمیل‌های هرزنامه و غیر هرزنامه:")
print(df['label'].value_counts())

# نمایش چند نمونه از متن ایمیل
print("\nنمونه هایی از متن ایمیل:")
for i in range(3):  # نمایش 3 نمونه
    print(f"\nنمونه {i+1} (برچسب: {df['label'][i]}):")
    print(df['text'][i][:200], "...")  # نمایش 200 کاراکتر اول هر ایمیل

نمایش چند ردیف اول دیتافریم ترکیب شده:
                                                text  label
0  mv 1 00001.bfc8d64d12b325ff385cca8d07b84288\nm...      1
1  From 12a1mailbot1@web.de  Thu Aug 22 13:17:22 ...      1
2  From ilug-admin@linux.ie  Thu Aug 22 13:27:39 ...      1
3  From sabrina@mx3.1premio.com  Thu Aug 22 14:44...      1
4  From wsup@playful.com  Thu Aug 22 16:17:00 200...      1

اطلاعات کلی دیتافریم ترکیب شده:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3052 entries, 0 to 3051
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3052 non-null   object
 1   label   3052 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.8+ KB

تعداد ایمیل‌های هرزنامه و غیر هرزنامه:
label
0    2551
1     501
Name: count, dtype: int64

نمونه هایی از متن ایمیل:

نمونه 1 (برچسب: 1):
mv 1 00001.bfc8d64d12b325ff385cca8d07b84288
mv 10 00010.7f5fb525755c45eb78efc18d7c9ea5aa
mv 100 00100.c60d1c697136b07c947fa180ba

---
# تبدیل ایمیل‌ها به ویژگی (feature extraction)
#### ما نمی‌تونیم متن خام رو مستقیماً به مدل بدیم. پس باید تبدیلش کنیم به عددی:

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
#می‌گوید که کلمات توقف (stop words) زبان انگلیسی را نادیده بگیرد.مانند "the"، "a"، "is"، "are" و غیره. با حذف این کلمات، تمرکز مدل بر روی کلمات مهم‌تر خواهد بود.

#فقط ۱۰۰۰ کلمه‌ی مهم‌تر و پرتکرارتر را از بین تمام کلمات موجود در همه ایمیل‌ها نگه دار و بقیه را نادیده بگیر

X = vectorizer.fit_transform(spam_emails + ham_emails)
y = [1]*len(spam_emails) + [0]*len(ham_emails)
#فرض کن 3 ایمیل اسپم داریم
# اون‌وقت len(spam_emails) می‌شه 3. حالا: [1] * 3
# مساوی می‌شه با: [1, 1, 1]
# # یعنی برای ۳ ایمیل اسپم، ۳ عدد ۱ تولید کردیم. همین کار برای ایمیل‌های معمولی:
# [0] * len(ham_emails)
# در نتیجه:
# y = [1, 1, 1, 0, 0, 0]


---
# آموزش مدل

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# تا اینجا چی ساختیم؟
# مثل اینه که یه ماشین داری که وقتی یه ایمیل بهش می‌دی، بهت می‌گه "این اسپمه یا نه؟"
# و برای این‌کار، خودش اول متن رو عددی می‌کنه، بعد مدل یادگرفته‌ش رو روی اون اجرا می‌کنه.

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       497
           1       0.99      0.88      0.93       114

    accuracy                           0.98       611
   macro avg       0.98      0.94      0.96       611
weighted avg       0.98      0.98      0.97       611

